In [1]:
# Import Libraries
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import pandas_datareader.data as web

S&P 500 Data

In [2]:
start_date = '2021-01-31'
end_date = '2022-12-31'
SnP_500_data = yf.download('^GSPC', start = start_date, end = end_date)
SnP_500_data['Adj Return'] = SnP_500_data['Adj Close'].pct_change()
SnP_500_data['Rolling_Vol.'] = SnP_500_data['Adj Return'].rolling(window=10).std()
SnP_500_data.drop(['Open', 'High', 'Low', 'Close', 'Volume', 'Adj Return'], inplace = True, axis = 1)
SnP_500_data.dropna(inplace = True)

[*********************100%%**********************]  1 of 1 completed


3-Month Treasury Yield Data

In [3]:
T3 = web.DataReader('DTB3','fred',start_date,end_date)

Merge S&P and Treasury Data

In [4]:
SnP_500_data = pd.merge(SnP_500_data, T3, left_index=True, right_index=True, how='inner')
SnP_500_data.rename(columns={'DTB3':'Risk-free'}, inplace=True)

In [5]:
max(SnP_500_data['Adj Close'])

4796.56005859375

Possible Strike

In [6]:
possible_strike = list(range(2300,5400 + 1, 25))
possible_strike = np.array(possible_strike)

In [7]:
SnP_500_data['Time To Maturity'] = 1/12
SnP_500_data['Price Option'] = 0
SnP_500_data = SnP_500_data.groupby(pd.Grouper(freq='MS')).first()
SnP_500_data

,Adj Close,Rolling_Vol.,Risk-free,Time To Maturity,Price Option
2021-02-01,3932.590088,0.005133,0.04,0.083333,0
2021-03-01,3901.820068,0.012452,0.05,0.083333,0
2021-04-01,4019.870117,0.007708,0.02,0.083333,0
2021-05-01,4192.660156,0.006999,0.04,0.083333,0
2021-06-01,4202.040039,0.005704,0.02,0.083333,0
2021-07-01,4319.939941,0.006841,0.05,0.083333,0
2021-08-01,4387.160156,0.006608,0.05,0.083333,0
2021-09-01,4524.089844,0.004736,0.05,0.083333,0
2021-10-01,4357.040039,0.011546,0.04,0.083333,0
2021-11-01,4613.669922,0.004142,0.05,0.083333,0


4D Array

In [8]:
# Define the dimensions of your 4D array

num_call_or_put = ['Call','Put'] # Call and Put
num_strike_prices = possible_strike

for j, val in enumerate(num_strike_prices):
    SnP_500_data['Strike'] = np.array([val] * SnP_500_data.shape[0])
    SnP_500_data['Profit/Loss Options'] = np.array([0]*SnP_500_data.shape[0])
    SnP_500_data['Profit/Loss Spot'] = np.array([0]*SnP_500_data.shape[0])
    
ind = SnP_500_data.index
col = SnP_500_data.columns

num_rows, num_cols = SnP_500_data.shape

four_dimensional_array = np.zeros((len(num_call_or_put), len(num_strike_prices), num_rows, num_cols), dtype=object)

for i in range(len(num_call_or_put)):
    for j,val in enumerate(num_strike_prices):
        SnP_500_data['Strike'] = np.array([val] * num_rows)
        four_dimensional_array[i,j,:,:] = SnP_500_data

Profit and Loss Calculation

In [9]:
def calculate_pnl(df,option_type):
    if option_type == 'Call':
        df['Profit/Loss Options'] = -df['Price Option'].diff().fillna(0) * 100
    else:
        df['Profit/Loss Options'] = -df['Price Option'].diff().fillna(0) * 100
    df['Profit/Loss Spot'] = df['Adj Close'].diff().fillna(0) * 100
    return df

Black-Scholes Calculation

In [10]:
def blackscholes(df,option_type):
    S_t = df['Adj Close']
    K = df['Strike']
    r = df['Risk-free']
    sigma = df['Rolling_Vol.']
    T_t = df['Time To Maturity']
    
    def d1(S_t, K, r, sigma, T_t):
        return (np.log(S_t / K) + (r + 0.5 * (sigma ** 2) * T_t) / (sigma * np.sqrt(T_t)))
    
    def d2(d1, sigma, T_t):
        return d1 - sigma * np.sqrt(T_t)
    
    # Calculate d1, d2, and the Black-Scholes option price for each row in the dataframe
    if option_type == 'Call':
        df['d1'] = d1(S_t, K, r, sigma, T_t)
        df['d2'] = d2(df['d1'], sigma, T_t)
        df['Price Option'] = S_t * norm.cdf(df['d1']) - K * np.exp(-r * T_t) * norm.cdf(df['d2'])
        df['Delta'] = norm.cdf(df['d1'])
    elif option_type == 'Put':
        df['d1'] = d1(S_t, K, r, sigma, T_t)
        df['d2'] = d2(df['d1'], sigma, T_t)
        df['Price Option'] = K * np.exp(-r * T_t) * norm.cdf(-df['d2']) - S_t * norm.cdf(-df['d1'])  
        df['Delta'] = -norm.cdf(df['d1'])
    else:
        raise ValueError("Invalid option_type. Use 'Call' or 'Put'.")
    
    df = calculate_pnl(df, option_type)
    df = df.drop(['d1', 'd2'], axis=1)
    return df

Create an Excel Writer Object

In [11]:
option_type = ('Call', 'Put')

for i, opt in enumerate(option_type):
    excel_writer = pd.ExcelWriter(opt + '.xlsx', engine = 'xlsxwriter')
    for j,val in enumerate(num_strike_prices):
        df = pd.DataFrame(four_dimensional_array[i,j,:,:], index= ind, columns = col)
        df = df.astype(float)
        df = blackscholes(df, opt)
        df.to_excel(excel_writer, sheet_name=str(val), index = True)
    excel_writer.close()

Output Excel Function

In [12]:
from openpyxl import load_workbook
def output(profit_loss_pf,call_pct, put_pct):
    option_type = ('Call', 'Put')
    for i, opt in enumerate(option_type):
        excel_writer = pd.ExcelWriter(opt + '.xlsx', engine = 'xlsxwriter')
        for j,val in enumerate(num_strike_prices):
            df = pd.DataFrame(four_dimensional_array[i,j,:,:], index= ind, columns = col)
            df = df.astype(float)
            df = blackscholes(df, opt)
            df.to_excel(excel_writer, sheet_name=str(val), index = True)
        excel_writer.close()
    excel_file_path = 'Call.xlsx'
    arr = []
    for row in profit_loss_pf.iterrows():
        call_strike = str(int(row[1][1]))
        df = pd.read_excel(excel_file_path, sheet_name=call_strike)
        entry = df[df.index == row[0]]['Profit/Loss Options'].iloc[0]
        arr.append(entry)
    arr = pd.DataFrame({'Profit/Loss': arr}, index=profit_loss_pf['Date'])
    profit_loss_pf = pd.merge(profit_loss_pf, arr, left_on='Date', right_index=True, how='inner')
    
    excel_file_path = 'Put.xlsx'
    arr = []
    for row in profit_loss_pf.iterrows():
        put_strike = str(int(row[1][2]))
        df = pd.read_excel(excel_file_path, sheet_name=put_strike)
        entry = df[df.index == row[0]]['Profit/Loss Options'].iloc[0]

        arr.append(entry)
    arr = pd.DataFrame({'Profit/Loss': arr}, index=profit_loss_pf['Date'])
    profit_loss_pf = pd.merge(profit_loss_pf, arr, left_on='Date', right_index=True, how='inner')
    
    excel_output_path = 'profit_loss_output_' + str(call_pct) +'C_'+ str(put_pct) +'P.xlsx'
    profit_loss_pf.to_excel(excel_output_path, index = False)

Profit / Loss Calculation

In [13]:
profit_loss_pf = pd.DataFrame(columns=['Date','Call_Strike','Put_Strike','Profit/Loss'])
profit_loss_pf['Date'] = SnP_500_data.index
profit_loss_pf = pd.merge(profit_loss_pf, SnP_500_data['Adj Close'], left_on='Date', right_index=True, how='inner')

3% Call, 5% Put

In [14]:
profit_loss_pf['Call_Strike'] = np.ceil(profit_loss_pf['Adj Close'] * 1.03 / 25) * 25
profit_loss_pf['Put_Strike'] = np.ceil(profit_loss_pf['Adj Close'] * 1.05 / 25) * 25
output(profit_loss_pf,3,5)

3% Call, 7% Put

In [15]:
profit_loss_pf['Call_Strike'] = np.ceil(profit_loss_pf['Adj Close'] * 1.03 / 25) * 25
profit_loss_pf['Put_Strike'] = np.ceil(profit_loss_pf['Adj Close'] * 1.07 / 25) * 25
output(profit_loss_pf,3,7)

6% Call, 1% Put

In [16]:
profit_loss_pf['Call_Strike'] = np.ceil(profit_loss_pf['Adj Close'] * 1.06 / 25) * 25
profit_loss_pf['Put_Strike'] = np.ceil(profit_loss_pf['Adj Close'] * 1.01 / 25) * 25
output(profit_loss_pf,6,1)

5% Call, 3% Put

In [17]:
profit_loss_pf['Call_Strike'] = np.ceil(profit_loss_pf['Adj Close'] * 1.06 / 25) * 25
profit_loss_pf['Put_Strike'] = np.ceil(profit_loss_pf['Adj Close'] * 1.01 / 25) * 25
output(profit_loss_pf,6,1)